# Importing Dataset

In [1]:
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pk
import json

In [2]:
df = pl.read_csv('./data/final_dataset.csv')
df.head()

sentiment,review
i64,str
1,"""I'm reading a lot of reviews s…"
1,"""This soundtrack is my favorite…"
1,"""I truly like this soundtrack a…"
1,"""If you've played the game, you…"
1,"""I am quite sure any of you act…"


# Preprocessing

In [3]:
import re
from nltk.corpus import stopwords

In [4]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [5]:
def preprocess_text(text):
    # remove tags and convert to lowercase
    sentence = remove_tags(text).lower()

    # remove punctuation, numbers, and single characters, and collapse multiple spaces
    sentence = re.sub(r'[^a-z\s]', ' ', sentence)  # remove non-alphabetic characters
    sentence = re.sub(r'\b[a-z]\b', '', sentence)  # remove single characters
    sentence = re.sub(r'\s+', ' ', sentence).strip()  # collapse multiple spaces

    # remove stopwords
    stop_words = set(stopwords.words('english'))
    sentence = ' '.join([word for word in sentence.split() if word not in stop_words])
    
    return sentence


In [6]:
# X = []
# sentences = list(df['review'])

# for sen in sentences:
#     X.append(preprocess_text(sen))
    
# with open('./data/preprocessed_text.txt', 'wb') as f:
#     pk.dump(X, f)

In [7]:
with open('./data/preprocessed_text.txt', 'rb') as f:
    X = pk.load(f)

In [8]:
X[2]

'truly like soundtrack enjoy video game music played game music enjoy truly relaxing peaceful disk one favorites scars time life death forest illusion fortress ancient dragons lost fragment drowned valley disk two draggons galdorb home chronomantique prisoners fate gale girlfriend likes zelbessdisk three best three garden god chronopolis fates jellyfish sea burning orphange dragon prayer tower stars dragon god radical dreamers unstealable jewel overall excellent soundtrack brought like video game music xander cross'

In [9]:
from sklearn.model_selection import train_test_split

y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preparing Embedding Layer

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-09-20 13:12:54.744132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 13:12:54.756153: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 13:12:54.759867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 13:12:54.770289: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 13:12:56.019007: W tensorflow/compiler/tf2

In [12]:
# word_tokenizer = Tokenizer()
# word_tokenizer.fit_on_texts(X_train)

# X_train = word_tokenizer.texts_to_sequences(X_train)
# X_test = word_tokenizer.texts_to_sequences(X_test)

In [13]:
# with open('./data/tokenizer.pkl', 'wb') as f:
#     pk.dump(word_tokenizer, f)

In [14]:
# tokenised_data = {
#     'X_train' : X_train,
#     'y_train' : y_train.to_list(),
#     'X_test' : X_test,
#     'y_test' : y_test.to_list()
# }

# with open('./data/tokenized_data.json', 'w') as f:
#     json.dump(tokenised_data, f)

In [15]:
with open('./data/tokenized_data.json', 'r') as f:
    tokenised_data = json.load(f)
    
X_train = tokenised_data['X_train']
X_test = tokenised_data['X_test']
y_train = tokenised_data['y_train']
y_test = tokenised_data['y_test']

In [16]:
y_train, y_test = np.array(y_train), np.array(y_test)

In [17]:
# vocab_length = len(word_tokenizer.word_index) + 1
# vocab_length
vocab_length = 779656

In [18]:
maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [19]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [20]:
# embeddings_dict = dict()

# with open('./data/glove.6B.100d.txt',  'r') as glove_file:
#     for line in glove_file:
#         records = line.split()
#         word = records[0]
#         vector_dimentions = np.asarray(records[1:], dtype='float32')
#         embeddings_dict[word] = vector_dimentions

In [21]:
# embeddings_dict

In [22]:
# with open('./data/word_tokenizer_items.txt', 'w') as f:
#     pk.dump(word_tokenizer.word_index.items(), f)

In [23]:
# embedding_matrix = np.zeros((vocab_length, 100))
# for word, index in word_tokenizer.word_index.items():
#     embedding_vector = embeddings_dict.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[index] = embedding_vector

In [24]:
# type(embedding_matrix), embedding_matrix[1, :]

In [25]:
# with open('./data/embedding_matrix.txt', 'wb') as f:
#     pk.dump(embedding_matrix, f)

In [26]:
with open('./data/embedding_matrix.txt', 'rb') as f:
    embedding_matrix = pk.load(f)

# LSTM Model

In [27]:
# !pip install keras-tuner

In [28]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Dropout, Embedding, LSTM, GRU, GlobalMaxPooling1D, BatchNormalization
from keras.models import Sequential
from keras_tuner import RandomSearch

In [29]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

I0000 00:00:1726818206.838890   10904 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726818206.998465   10904 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726818206.998734   10904 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [30]:
def build_lstm_model(hp):
    model = Sequential([
        Embedding(vocab_length,
                  100,
                  weights=[embedding_matrix],
                  input_length=maxlen,
                  trainable=False),
        
        LSTM(hp.Int('lstm_units', min_value=64, max_value=256, step=32)),
        
        Dense(1, activation='sigmoid')
    ])


    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    return model

In [31]:
tuner = kt.RandomSearch(
    build_lstm_model,            
    objective='val_accuracy',    
    max_trials=10,               
    executions_per_trial=1,      
    directory='./tuning_dir',   
    project_name='lstm_tuning'   
)

Reloading Tuner from ./tuning_dir/lstm_tuning/tuner0.json


I0000 00:00:1726818207.030104   10904 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726818207.030640   10904 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726818207.030973   10904 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726818207.117118   10904 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [32]:
# tuner.search(X_train_, y_train_, 
#              epochs=5, 
#              validation_data=(X_val, y_val),
#              batch_size=32)

In [33]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

2024-09-20 13:13:31.284606: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.
2024-09-20 13:13:32.721118: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │    77,965,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 192)            │       225,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,190,817 (298.27 MB)

 Trainable params: 225,217 (879.75 KB)

 Non-trainable params: 77,965,600 (297.42 MB)

In [34]:
# with tf.device('/GPU:0'):  # Force the model to run on the GPU
#     lstm_model = Sequential([
#         Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False),
        
#         LSTM(128, return_sequences=True),
#         Dropout(0.3),
        
#         LSTM(128, return_sequences=True),  # Keep the output 3D
#         Dropout(0.3),
        
#         GlobalMaxPooling1D(),  # Now works as it has a 3D input
        
#         Dense(128, activation='relu'),
#         BatchNormalization(),
#         Dropout(0.3),
        
#         Dense(64, activation='relu'),
#         BatchNormalization(),
#         Dropout(0.3),
        
#         Dense(1, activation='sigmoid')
#     ])

#     # Compile the model (ensure loss is compatible with mixed precision)
#     lstm_model.compile(
#         optimizer='adam',
#         loss='binary_crossentropy',
#         metrics=['acc']
#     )

#     # Train the model
#     lstm_model_history = lstm_model.fit(
#         X_train,
#         y_train,
#         batch_size=128,
#         epochs=10,
#         verbose=1,
#         validation_data=(X_val, y_val)
#     )

In [35]:
# lstm_model.compile(
#     optimizer='adam',
#     loss='binary_crossentropy',
#     metrics=['acc']
# )

In [36]:
# score = best_model.evaluate(X_test, y_test, verbose=1)

# print(f'Test Score : {score[0]}\nTest Accuracy: {score[1]}')

In [37]:
# score = lstm_model.evaluate(X_test, y_test, verbose=1)

# print(f'Test Score : {score[0]}\nTest Accuracy: {score[1]}')

In [38]:
# lstm_model.save('./models/lstm_model.h5')

In [39]:
saved_model = tf.keras.models.load_model('./models/lstm_model.h5')

2024-09-20 13:13:33.982277: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.


In [40]:
saved_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 100)       │    77,965,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100, 128)       │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100, 128)       │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,240,035 (298.46 MB)

 Trainable params: 274,049 (1.05 MB)

 Non-trainable params: 77,965,984 (297.42 MB)

 Optimizer params: 2 (12.00 B)

In [41]:
# score = saved_model.evaluate(X_test, y_test, verbose=1)

# print(f'Test Score : {score[0]}\nTest Accuracy: {score[1]}')

In [42]:
# Define the hypermodel that takes hyperparameters as input
def build_model(hp):
    with tf.device('/GPU:0'):  # Force the model to run on the GPU
        model = Sequential()

        # Embedding Layer (kept the same, but you can also tune if necessary)
        model.add(Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False))
        
        # First LSTM layer
        model.add(LSTM(
            units=hp.Int('lstm_units_1', min_value=64, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

        # Second LSTM layer
        model.add(LSTM(
            units=hp.Int('lstm_units_2', min_value=64, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

        # GlobalMaxPooling1D
        model.add(GlobalMaxPooling1D())

        # Dense layers
        model.add(Dense(
            units=hp.Int('dense_units_1', min_value=64, max_value=256, step=32),
            activation='relu'
        ))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))

        model.add(Dense(
            units=hp.Int('dense_units_2', min_value=32, max_value=128, step=16),
            activation='relu'
        ))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout_4', min_value=0.1, max_value=0.5, step=0.1)))

        # Output Layer
        model.add(Dense(1, activation='sigmoid'))

        # Compile the model with a tunable learning rate
        model.compile(
            optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
            loss='binary_crossentropy',
            metrics=['acc']
        )

        return model


In [43]:
# Define the tuner for Random Search (other options are Hyperband and BayesianOptimization)
tuner = RandomSearch(
    build_model,
    objective='val_acc',  # Tuning based on validation accuracy
    max_trials=10,  # Number of different models to try
    executions_per_trial=1,  # Number of times to run each model
    directory='tuning_dir',  # Directory to save tuner results
    project_name='lstm_tuning_enhanced'
)

Reloading Tuner from tuning_dir/lstm_tuning_enhanced/tuner0.json


In [44]:
# Perform the hyperparameter search
# tuner.search(
#     X_train, 
#     y_train, 
#     epochs=5,  # Reduce epochs for faster tuning
#     batch_size=64,  # Set a default batch size (or tune it inside `build_model` if needed)
#     validation_data=(X_val, y_val),
#     verbose=1
# )

In [45]:
# Retrieve the best model
best_model_enhanced = tuner.get_best_models(num_models=1)[0]

# Summary of the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hyperparameters.values)

2024-09-20 13:13:38.265719: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.


Best Hyperparameters: {'lstm_units_1': 224, 'dropout_1': 0.1, 'lstm_units_2': 192, 'dropout_2': 0.2, 'dense_units_1': 128, 'dropout_3': 0.4, 'dense_units_2': 80, 'dropout_4': 0.5, 'learning_rate': 0.0004247676943610829}


2024-09-20 13:13:39.228495: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.


In [46]:
best_model_enhanced.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │    77,965,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 224)       │       291,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 224)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 192)       │       320,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 192)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 192)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 80)             │        10,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 80)             │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,612,993 (299.88 MB)

 Trainable params: 646,977 (2.47 MB)

 Non-trainable params: 77,966,016 (297.42 MB)

In [47]:
# score = best_model_enhanced.evaluate(X_test, y_test, verbose=1)

# print(f'Test Score : {score[0]}\nTest Accuracy: {score[1]}')

In [48]:
# best_model_enhanced.save('./models/lstm_model_enhanced.h5')

In [61]:
model = tf.keras.models.load_model('./models/lstm_model_enhanced.h5')

In [50]:
with open('./data/tokenizer.pkl', 'rb') as f:
    tokenizer = pk.load(f)

In [83]:
def model_predict(text, tokenizer, model):
    clean_text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([clean_text])
    padded_sequence = pad_sequences(sequence, maxlen=100)
    prediction = model.predict(padded_sequence)
    prediction = 'Negative' if prediction< 0.5 else 'Positive'
    
    return prediction

In [86]:
text = "This DVD will be a disappointment if you get it hoping to see some substantial portion of the acts of the various comics listed on the cover. All you get here are snippets of performance, at best. The rest is just loose-leaf reminiscence about the good old days in Boston, in the early 80's, when a lot of comics were hanging out together and getting their start.It's like a frat house reunion. There's a lot of lame nostalgia. There are quite a few guffaws recalling jokes (practical and otherwise)perpetrated - back then. But you had to have been there to appreciate all the basically good ol' boy camaraderie. If you weren't actually a part of that scene, all this joshing and jostling will fall flat.If you want to actually hear some of these comics' routines - you will have to look elsewhere."

pred = model_predict(text=text, tokenizer=tokenizer, model=model)
print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Negative


: 